In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import math


In [28]:
data_id = '20201022a.pfb'
#data_id = '20201004c.tb'

In [29]:
#import tb
#tb = pd.read_csv('real-time-data.202001-20200331.TB.curves.20201015d.all-bonds-comparison.csv',names = ['fitTime','id','maturityDate','a','b','c','d','e','f','issueDate','yield','h','i'],engine='python')
#tb = pd.read_csv('data/20201004c.tb/5502.curves.20201004c.all-bonds-comparison_original.csv', names = ['fitTime','id','maturityDate','a','b','c','d','e','f','issueDate','yield','h','i'],engine='python')
#cdb = pd.read_csv('real-time-data.20200106-20200605.CDB.curves.20201022a.all-bonds-comparison.csv',names = ['fitTime','id','maturityDate','a','b','c','d','e','f','issueDate','yield','h','i'],engine='python')
pfb = pd.read_csv('real-time-data.202001-20200929.PFB.curves.20201022a.all-bonds-comparison.csv',names = ['fitTime','id','maturityDate','a','b','c','d','e','f','issueDate','yield','h','i'],engine='python')

In [30]:
test_bond_type = 'PFB'
if test_bond_type =='TB':
    source_data = tb
elif test_bond_type == 'CDB':
    source_data = cdb
elif test_bond_type == 'PFB':
    source_data = pfb

In [31]:

def adjust_utf8(x):
    try:
        float(x['yield'])
    except ValueError:
        return float(x['issueDate'])
    else:
        return float(x['yield'])
source_data['yield'] = source_data.apply(lambda x:adjust_utf8(x),axis = 1)


In [32]:
source_data = source_data[['fitTime','id','maturityDate','yield']]
source_data.insert(loc = 2, column= 'TYPE',value='theo')
source_data.insert(loc = 3, column = 'bond_type', value=test_bond_type)


In [33]:
#import trade_data
trade_data = pd.read_excel('Jan_Oct_5502.xlsx')
#encoding = 'GB2312'

def set_bond_type_trade_data(id):
    if id[2:4]=='00' or id[2:4]=='99':
        return 'TB'
    elif id[2:4]=='02' or id[2:4]=='77':
        return 'CDB'
    else:
        return 'PFB'

In [34]:
def set_0_in_same_time_bucket(x):
    for i in range(x.shape[0]-1):
        if x.iloc[i,2] == 5502 and x.iloc[i+1,2] == 5502:
            x.iloc[i,2] = 0
            
            
def get_time_diff(x):
    for i in range(1, len(x.index)):
        if x.iloc[i, 7] == x.iloc[i - 1, 7]:
            x.iloc[i, 8] = x.iloc[i, 0] - x.iloc[i - 1, 0]
        else:
            x.iloc[i, 8] = x.iloc[i, 0] - datetime.datetime.combine(x.iloc[i, 7], datetime.time(6, 52, 0))

    for i in range(1, len(x.index)):
        x.iloc[i, 8] = x.iloc[i, 8].total_seconds()
        
        
def calculate_insert(x):
    x.insert(x.shape[1], 'yield_change', 0)
    x.insert(x.shape[1], 'date', 0)
    x.insert(x.shape[1], 'time_diff', 0)




def get_date_part(x):
    return x.date()


        
def Java_time_to_datetime(s):
    date = s[0:10]
    time = s[11:]
    if len(time) == 8:   
        new = datetime.datetime.strptime(date+time,'%Y-%m-%d%H:%M:%S')
    elif len(time) == 5:
        new = datetime.datetime.strptime(date+time,'%Y-%m-%d%H:%M')
    elif len(time) ==2:
        new = datetime.datatime.strptime(date+time,'%Y-%m-%d%H')
    return new
    

In [35]:
#normalize source_data
source_data['fitTime'] = source_data['fitTime'].apply(lambda x :Java_time_to_datetime(x))
source_data['maturityDate'] = source_data['maturityDate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d'))

#date from 2020-01-02 to 2020-06-05
#source_data = source_data[source_data['fitTime']<datetime.datetime(year = 2020, month = 6, day = 6)]

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
#normalize trade_data
trade_data.rename(columns={'time':'fitTime','trade_id':'id','MD_entry_px':'yield','maturity_date':'maturityDate'},inplace=True)
trade_data = trade_data[['fitTime','id','TYPE','bond_type','maturityDate','yield']]
trade_data['bond_type'] = trade_data['id'].apply(lambda x :set_bond_type_trade_data(x))
trade_data['yield']/=100
trade_data = trade_data[trade_data['bond_type']==test_bond_type]

#trade_data time limit
trade_data = trade_data[trade_data['fitTime']<datetime.datetime(year = 2020, month = 4, day = 1)]

In [37]:
day_list = trade_data['fitTime'].apply(lambda x:datetime.datetime.date(x)).unique().tolist()

In [38]:
drop_bond_list = []

In [39]:
#bond_catagory = pd.read_csv('data/'+data_id+'/bond_type_each_day_'+data_id+'.csv')

In [40]:
bond_catagory = pd.read_csv('bond_type_each_day_'+data_id+'.csv')

In [41]:
bond_catagory['date'] = bond_catagory['date'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d'))  
bond_catagory['date'] = bond_catagory['date'].apply(lambda x: get_date_part(x))

In [42]:
bond_catagory[bond_catagory['liquidity_type']=='high']

,id,date,maturity_type,liquidity_type
33,180304.IB,2020-01-07,mid,high
34,190406.IB,2020-01-07,midlong,high
35,190405.IB,2020-01-07,short,high
36,190307.IB,2020-01-07,short,high
46,180304.IB,2020-01-08,mid,high
...,...,...,...,...
2757,190307.IB,2020-09-25,short,high
2759,200401.IB,2020-09-25,short,high
2780,160403.IB,2020-09-29,short,high
2781,180411.IB,2020-09-29,midlong,high


In [43]:
predicting_error_table = pd.DataFrame(columns = ['fitTime','id','TYPE','bond_type','maturityDate','yield','real','theo','predicting_error',
                                                'real_error','date','maturity_type','liquidity_type'])

In [44]:
selected_bond_list = trade_data['id'].unique().tolist()
for bond_id in selected_bond_list:
    
    new_source_data = source_data[source_data['id'] == bond_id]
    new_trade_data = trade_data[trade_data['id'] == bond_id]
    
    #concat
    merged_table = pd.concat([new_source_data,new_trade_data])
    merged_table = merged_table.sort_values(by=['fitTime','TYPE'])

    merged_table.reset_index(drop = True)
    
    set_0_in_same_time_bucket(merged_table)
    merged_table
    
    merged_table['TYPE'] = merged_table['TYPE'].replace(0,np.nan)
    merged_table = merged_table.dropna()
    
    merged_table = merged_table.reset_index(drop = True)
    
    merged_table['real'] = 0
    merged_table['theo'] = 0
    merged_table['predicting_error'] = 0
    
    if merged_table.iloc[0,2] == 5502:
        merged_table.iloc[0,6] = merged_table.iloc[0,5]
    else:
        pass

    for i in range(1,merged_table.shape[0]):
        if merged_table.iloc[i,2] == 5502 and merged_table.iloc[i-1,2] == 'theo':
            merged_table.iloc[i,6] = merged_table.iloc[i,5]
            merged_table.iloc[i,7] = merged_table.iloc[i-1,5]
            merged_table.iloc[i,8] = merged_table.iloc[i,7] - merged_table.iloc[i,6]

    merged_table['real']  = merged_table['real'].replace(0,np.nan)
    merged_table = merged_table.dropna()

    merged_table = merged_table.reset_index(drop = True)

    merged_table['real_error'] = merged_table['real'].diff()
    merged_table = merged_table.dropna()
    merged_table['date'] = merged_table['fitTime'].apply(lambda x: get_date_part(x))    
    merged_table =  pd.merge(merged_table,bond_catagory, on = ['id','date'])
    
    predicting_error_table = pd.concat([predicting_error_table,merged_table])


    

In [45]:
predicting_error_table = predicting_error_table.sort_values(by=['fitTime'])

In [46]:
predicting_error_table = predicting_error_table.reset_index(drop = True)

In [47]:
maturity_list = ['short','mid','midlong','long']
liquidity_list = ['low','high']

In [48]:
PE_MPE = pd.DataFrame(columns= ['count','Avg(theo-real)','Max','Min','5% percentile','95% percentile','Abs_Avg','Abs_Max','Abs_95% percentile'])
PE_RE = pd.DataFrame(columns= ['count','Avg(theo-real)','Max','Min','5% percentile','95% percentile','Abs_Avg','Abs_Max','Abs_95% percentile'])

In [49]:
for i in maturity_list:
    for j in liquidity_list:
        bond_data_final = predicting_error_table[predicting_error_table['maturity_type'] == i]
        bond_data_final = bond_data_final[bond_data_final['liquidity_type'] == j]
        PE = bond_data_final['predicting_error']
        RE = bond_data_final['real_error']
        
        if len(bond_data_final)==0:
            PE_MPE.loc['PE_MPE_'+bond_type_for_test+'_'+i+'_'+j] = pd.Series({'count':0,'Avg(theo-real)':'N/A','Max':'N/A','Min':'N/A','5% percentile':
                             'N/A', '95 percentile':'N/A','Abs_Avg':'N/A','Abs_Max':'N/A','Abs_95% percentile':'N/A'})
            PE_RE.loc['RE_'+bond_type_for_test+'_'+i+'_'+j] = pd.Series({'count':0,'Avg(theo-real)':'N/A','Max':'N/A','Min':'N/A','5% percentile':
                             'N/A', '95 percentile':'N/A','Abs_Avg':'N/A','Abs_Max':'N/A','Abs_95% percentile':'N/A'})
        else:
            MPE = pd.Series({'count':len(PE),'Avg(theo-real)':np.nanmean(PE),'Max':max(PE),'Min':min(PE),
                          '5% percentile':np.percentile(PE,5),'95% percentile':np.percentile(PE,95),
                          'Abs_Avg':np.mean(abs(PE)),'Abs_Max':max(abs(PE))
                          ,'Abs_95% percentile':np.percentile(abs(PE),95)})
            RPE = pd.Series({'count':len(RE),'Avg(theo-real)':np.nanmean(RE),'Max':max(RE),'Min':min(RE),
                          '5% percentile':np.percentile(RE,5),'95% percentile':np.percentile(RE,95),
                          'Abs_Avg':np.mean(abs(RE)),'Abs_Max':max(abs(RE))
                          ,'Abs_95% percentile':np.percentile(abs(RE),95)})
            PE_MPE.loc['PE_MPE_'+test_bond_type+'_'+i+'_'+j] = MPE
            PE_RE.loc['RE_'+test_bond_type+'_'+i+'_'+j] = RPE



In [50]:
PE_MPE.to_csv('PE_MPE_bond_maturity_liquidity_'+data_id+'.csv')

In [51]:
PE_RE.to_csv('PE_RE_bond_maturity_liquidity_'+data_id+'.csv')

In [52]:
PE_MPE

,count,Avg(theo-real),Max,Min,5% percentile,95% percentile,Abs_Avg,Abs_Max,Abs_95% percentile
PE_MPE_PFB_short_low,122.0,0.000351,0.016813,-0.013648,-0.007449,0.006824,0.002442,0.016813,0.009704
PE_MPE_PFB_short_high,28.0,0.000218,0.004100,-0.001900,-0.000500,0.001493,0.000525,0.004100,0.001893
PE_MPE_PFB_mid_low,388.0,0.002342,0.042155,-0.010685,-0.004317,0.023763,0.003598,0.042155,0.023763
PE_MPE_PFB_mid_high,153.0,0.000068,0.002194,-0.003053,-0.000360,0.000920,0.000313,0.003053,0.001483
PE_MPE_PFB_midlong_low,617.0,0.000075,0.005195,-0.001383,-0.000947,0.002484,0.000584,0.005195,0.002484
PE_MPE_PFB_midlong_high,333.0,0.000117,0.001767,-0.001096,-0.000210,0.000682,0.000196,0.001767,0.000709
PE_MPE_PFB_long_low,4.0,0.001464,0.003776,0.000200,0.000269,0.003392,0.001464,0.003776,0.003392
PE_MPE_PFB_long_high,9.0,0.000720,0.003675,-0.004167,-0.002796,0.003529,0.001988,0.004167,0.003970


In [60]:
predicting_error_table

,fitTime,id,TYPE,bond_type,maturityDate,yield,real,theo,predicting_error,real_error,date,maturity_type,liquidity_type
0,2020-01-02 09:26:08,190005.IB,5502,TB,2020-05-23,0.022000,0.022000,0.021946,-0.000054,0.000000,2020-01-02,short,low
1,2020-01-02 09:52:34,190006.IB,5502,TB,2029-05-23,0.031625,0.031625,0.031602,-0.000023,0.000025,2020-01-02,midlong,low
2,2020-01-02 10:02:51,190006.IB,5502,TB,2029-05-23,0.031625,0.031625,0.031566,-0.000059,0.000000,2020-01-02,midlong,low
3,2020-01-02 10:08:26,190003.IB,5502,TB,2022-03-07,0.026400,0.026400,0.026115,-0.000285,-0.000100,2020-01-02,mid,low
4,2020-01-02 10:44:22,190007.IB,5502,TB,2026-06-06,0.030500,0.030500,0.030465,-0.000035,0.000000,2020-01-02,midlong,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3244,2020-03-31 17:01:46,190015.IB,5502,TB,2029-11-21,0.025800,0.025800,0.025900,0.000100,-0.000025,2020-03-31,midlong,high
3245,2020-03-31 17:08:53,190006.IB,5502,TB,2029-05-23,0.026475,0.026475,0.025884,-0.000591,-0.000025,2020-03-31,midlong,low
3246,2020-03-31 17:10:10,190010.IB,5502,TB,2049-07-22,0.032550,0.032550,0.032594,0.000044,-0.000025,2020-03-31,long,high
3247,2020-03-31 17:20:33,190010.IB,5502,TB,2049-07-22,0.032550,0.032550,0.032590,0.000040,0.000000,2020-03-31,long,high


In [53]:
predicting_error_table = pd.DataFrame(columns = ['fitTime','id','TYPE','bond_type','maturityDate','yield','real','theo','predicting_error',
                                                'real_error','date','maturity_type','liquidity_type'])

In [30]:
merged_table

,fitTime,id,TYPE,bond_type,maturityDate,yield,real,theo,predicting_error,real_error,date,maturity_type,liquidity_type
0,2020-01-02 09:26:08,190005.IB,5502,TB,2020-05-23,0.0220,0.0220,0.021946,-5.398761e-05,0.0000,2020-01-02,short,low
1,2020-01-02 14:42:08,190005.IB,5502,TB,2020-05-23,0.0224,0.0224,0.022331,-6.920035e-05,0.0004,2020-01-02,short,low
2,2020-01-07 14:54:02,190005.IB,5502,TB,2020-05-23,0.0223,0.0223,0.022000,-3.000071e-04,-0.0001,2020-01-07,short,high
3,2020-01-08 13:55:00,190005.IB,5502,TB,2020-05-23,0.0222,0.0222,0.022000,-2.000057e-04,-0.0001,2020-01-08,short,high
4,2020-01-08 14:07:23,190005.IB,5502,TB,2020-05-23,0.0222,0.0222,0.022000,-2.000098e-04,0.0000,2020-01-08,short,high
5,2020-01-09 10:24:15,190005.IB,5502,TB,2020-05-23,0.0220,0.0220,0.022000,-3.696308e-08,-0.0002,2020-01-09,short,high
6,2020-01-13 15:46:16,190005.IB,5502,TB,2020-05-23,0.0219,0.0219,0.021964,6.389240e-05,-0.0001,2020-01-13,short,low
7,2020-01-14 11:24:58,190005.IB,5502,TB,2020-05-23,0.0220,0.0220,0.022000,-1.416697e-08,0.0001,2020-01-14,short,high
8,2020-02-11 10:43:12,190005.IB,5502,TB,2020-05-23,0.0165,0.0165,0.021650,5.150168e-03,-0.0055,2020-02-11,short,low
9,2020-02-11 13:34:21,190005.IB,5502,TB,2020-05-23,0.0165,0.0165,0.021721,5.220629e-03,0.0000,2020-02-11,short,low


In [ ]:
for i in maturity_list:
    for j in liquidity_list:
        bond_date_list = bond_catagory[bond_catagory['maturity_type'] == i]
        bond_date_list = bond_date_list[bond_date_list['liquidity_type'] == j]
        

        real_var_bucket_list = []
        real_vol_bucket_list = []
        theo_var_bucket_list = []
        theo_vol_bucket_list = []
        
        
        bond_date_list['date'] = bond_date_list['date'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d'))  
        bond_date_list['date'] = bond_date_list['date'].apply(lambda x: get_date_part(x))
        bond_date_list = bond_date_list.reset_index(drop = True)
        
        bond_date_tuples = []
        
        for k in range(len(bond_date_list.index)):
            bond_date_tuples.append((bond_date_list.iloc[k,0],bond_date_list.iloc[k,1]))

        for particular_bond, particular_date in bond_date_tuples:
            
            try:
                math.isnan(real_var_table[particular_date][particular_bond])
            except KeyError:
                continue
            else:
                pass
                
            
            if math.isnan(real_var_table[particular_date][particular_bond]):
                continue
            else:
                real_var_bucket_list.append(real_var_table[particular_date][particular_bond])
                real_vol_bucket_list.append(real_vol_table[particular_date][particular_bond])
                theo_var_bucket_list.append(theo_var_table[particular_date][particular_bond])
                theo_vol_bucket_list.append(theo_vol_table[particular_date][particular_bond])
                
                
            
                debug_real_var_bucket_list = debug_real_var_bucket_list.append(pd.Series({'date':particular_date,'id':particular_bond,'value':real_var_table[particular_date][particular_bond],'maturity_type':i,'liquidity_type':j}),ignore_index=True)
                debug_real_vol_bucket_list = debug_real_vol_bucket_list.append(pd.Series({'date':particular_date,'id':particular_bond,'value':real_vol_table[particular_date][particular_bond],'maturity_type':i,'liquidity_type':j}),ignore_index=True)
                debug_theo_var_bucket_list = debug_theo_var_bucket_list.append(pd.Series({'date':particular_date,'id':particular_bond,'value':theo_var_table[particular_date][particular_bond],'maturity_type':i,'liquidity_type':j}),ignore_index=True)
                debug_theo_vol_bucket_list = debug_theo_vol_bucket_list.append(pd.Series({'date':particular_date,'id':particular_bond,'value':theo_vol_table[particular_date][particular_bond],'maturity_type':i,'liquidity_type':j}),ignore_index=True)
                
                
                
                
                
                
                
                
                
        real_var_mean = np.mean(real_var_bucket_list)
        theo_var_mean = np.mean(theo_var_bucket_list)
        
        bucket_stats = pd.Series({'count':len(real_var_bucket_list),'real_var':real_var_mean,'real_vol':np.sqrt(real_var_mean)
                          ,'theo_var':theo_var_mean,'theo_vol':np.sqrt(theo_var_mean),
                                'ratio_var':theo_var_mean/real_var_mean,
                                'ratio_vol':np.sqrt(theo_var_mean)/np.sqrt(real_var_mean)
                                 })
        Var_Vol_Test.loc['maturity_type = '+i+'   and   '+'liquidity_type = '+j] = bucket_stats

In [ ]:
Var_Vol_Test.to_csv('Test_Result/'+data_id+'/Var_Vol_Test_result_'+data_id+'.csv')

In [ ]:
debug_real_var_bucket_list

In [ ]:
debug_real_var_bucket_list.to_csv('Test_Result/'+data_id+'/debug_real_var_bucket_list.csv')
debug_real_vol_bucket_list.to_csv('Test_Result/'+data_id+'/debug_real_vol_bucket_list.csv')
debug_theo_var_bucket_list.to_csv('Test_Result/'+data_id+'/debug_theo_var_bucket_list.csv')
debug_theo_vol_bucket_list.to_csv('Test_Result/'+data_id+'/debug_theo_vol_bucket_list.csv')

In [ ]:
Var_Vol_Test

In [ ]:
'''
bond = bond_type[bond_type['liquid'] !='Low']
bond_dict = {}
for i in range(bond.shape[0]):
    bond_dict[bond.loc[i,'id']] = [bond.loc[i,'liquid']]
for i in bond_dict:
    series = trade_data[trade_data['trade_id']==i].tail(1)
    series = series.reset_index(drop = True)
    bond_dict[i].append(series.loc[0,'bucket'])
bond_catagory = pd.DataFrame(bond_dict,index=['liquid','maturity_type'])
bond_catagory = bond_catagory.T
bond_catagory.reset_index(inplace = True)
bond_catagory['maturity'] = 0
for i in range(bond_catagory.shape[0]):
    if bond_catagory.at[i,'maturity_type']=='(0,1]':
        bond_catagory.loc[i,'maturity'] = 'short'
    elif bond_catagory.at[i,'maturity_type']=='(1,5]':
        bond_catagory.loc[i,'maturity'] = 'mid'
    elif bond_catagory.at[i,'maturity_type']=='(5,10]':
        bond_catagory.loc[i,'maturity'] = 'midlong'
    elif bond_catagory.at[i,'maturity_type']=='(10,+)':
        bond_catagory.loc[i,'maturity'] = 'long'
bond_catagory
'''
pass

In [ ]:
bond_catagory.drop(columns='Unnamed: 0',inplace=True)

In [ ]:
bond_catagory

In [ ]:
def get_bond(liquid = 'high', maturity = 'long'):
    res = bond_catagory[bond_catagory['liquidity_type']==liquid]
    res = res[res['maturity_type']==maturity]
    return res



In [ ]:
#excel_function:
#=IF(LEN(A1)=19,DATE(LEFT(A1,4),MID(A1,6,2),MID(A1,9,2))+TIME(MID(A1,12,2),MID(A1,15,2),RIGHT(A1,2)),DATE(LEFT(A1,4),MID(A1,6,2),MID(A1,9,2))+TIME(MID(A1,12,2),MID(A1,15,2),0))



    
def add_forward_to_maturity(data):
    data['forward_to_maturity']=0
    data.loc['forward_to_maturity'] = data['maturityDate'] - data['fitTime']
    data.loc['forward_to_maturity'] = data['forward_to_maturity'] / timedelta(days = 1)
    data['forward_to_maturity']/=365.25
    return data['forward_to_maturity']
    



In [ ]:
def set_date(data):
    return data.date()

def get_tail_data(data):
    selected_data = data.groupby('date').tail(1)
    return selected_data


In [ ]:
def get_yield(data):
    yield_dif = data['yield'].diff()
    return yield_dif

In [ ]:
def get_avg(data):
    time_dif = data['time'].diff()/timedelta(seconds = 1)
    time_dif[time_dif<10] = 10
    yield_dif = get_yield(data)
    avg = yield_dif / time_dif
    return avg

def get_var_one(data):
    time_dif = data['time'].diff()/timedelta(seconds = 1)
    time_dif[time_dif<10] = 10
    yield_dif = get_yield(data)
    var = yield_dif**2 / time_dif
    return var

def add_y0(data):
    i=0
    new_data = data.copy(deep = True)
    n = data.shape[0]
    list_of_tail = new_data.groupby('date').tail(1).index
    temp = 0
    for j in list_of_tail[:-1]:
        i = j+temp
        part1 = new_data.iloc[:i+1]
        part2 = new_data.iloc[i+1:]
        if i >= n:
            continue
        s = new_data.loc[i]
        new_day_first_time = part2.iloc[0,:]['time']
        new_date = new_day_first_time.date()
        if s['time'].date()<new_date-timedelta(days = 1) or s['time'].time() < time(hour = 19, minute = 0, second = 0):
            s.loc['time'] = new_date+timedelta(hours = 6, minutes = 52)
        else:
            gap_time = (datetime(new_date)+timedelta(hours = 8) - s['time'])/10       
            s.loc['time'] = (datetime(new_date)+timedelta(hours = 8) - gap_time)
            s.loc['time'] = s['time'].replace(microsecond = 0) 

        part1 = part1.append(s)
        new_data = part1.append(part2)
        new_data = new_data.reset_index(drop = True)
        temp+=1
    return new_data



def get_variance(data):
    var_diff_date = data.groupby('date').apply(get_var_one)
    var_diff_date_sum = var_diff_date.groupby('date').sum()
    buckets_n = var_diff_date.groupby('date').count()    
    variance_of_yield_changes =(var_diff_date_sum - var_daily.to_numpy())/(buckets_n-1)
    return variance_of_yield_changes

def adjust_first_day(data):
    var_daily = get_var_one(tail_data)
    date_start = data['date'][0]
    first_day = data[data['date']==date_start]
    time_dif_first_day = (first_day['fitTime'][len(first_day)-1] - first_day['fitTime'][0]) /timedelta(seconds = 1)
    var_daily[0] = (first_day[test_yield][len(first_day)-1]-first_day[test_yield][0])**2/ time_dif_first_day
    return var_daily

def rename_real_data(data):
    data.rename(columns = {'time':'fitTime'},inplace = True)




            

            
        
        
        
    
    

In [ ]:
tb

In [ ]:
trade_data

In [ ]:
tb['fitTime'] = tb['fitTime'].apply(lambda x :Java_time_to_datetime(x))
tb['maturityDate'] = tb['maturityDate'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))
tb['forward_to_maturity'] = 0
tb['forward_to_maturity'] = tb.apply(lambda x:add_forward_to_maturity(x),axis = 1)

In [ ]:
tb.to_csv('data/20201015d.tb/new_tb_vol_test.csv',index= False)

In [ ]:
tb = pd.read_csv('data/20201015d.tb/new_tb_vol_test.csv')

In [ ]:
bond_list = get_bond(liquid = 'mid', maturity= 'long')['id'].tolist()
bond_list

for i in bond_list:
    data_theo = theoretical_data[theoretical_data['id']==i]
    data_theo = data_theo.reset_index(drop = True)
    #Java_time_to_datetime(data_theo,'fitTime')
    #Java_time_to_datetime(data_theo,'maturity')

    

    set_date(IB_190006)

In [ ]:
trade_data

In [ ]:
tb.insert(loc = 1, column = 'date', value =0)
tb['date'] = tb['fitTime'].apply(lambda x: set_date(x))

In [ ]:
trade_data.insert(loc = 1, column = 'date', value =0)
trade_data['date'] = trade_data['time'].apply(lambda x: set_date(x))

In [ ]:
trade_data.rename(columns={'MD_entry_px':'yield'},inplace= True)

In [ ]:
trade_data['yield']/=100

In [ ]:
trade_data

In [ ]:
trade_data.to_csv('trade_data_a.csv')

In [ ]:
trade_data_TB = trade_data[trade_data['bond_type']=='TB']

for i in tb_bond_list:
    i = '190006.IB'
    bond_data_real = trade_data_TB[trade_data_TB['trade_id'] == i ]
    bond_data_real = bond_data_real.reset_index(drop = True)
    
    bond_data_real_with_y0 = add_y0(bond_data_real)
    var_daily = bond_data_real_with_y0.groupby('date').apply(lambda x : (x['yield'].tail(1) - x['yield'].head(1))/((x['time'].tail(1)-x['time'].head(1))/timedelta(seconds=1)))
    
    variance_of_yield_changes = get_variance(bond_data_real_with_y0)
    print(variance_of_yield_changes)
    volatility = variance_of_yield_changes**(1/2)
    break
    
    
    

In [ ]:

IB_190006 = trade_data_TB[trade_data_TB['trade_id'] == '190006.IB' ]

bond_data_real = trade_data_TB[trade_data_TB['trade_id'] == i ]
    bond_data_real = bond_data_real.reset_index(drop = True)
    
    bond_data_real_with_y0 = add_y0(bond_data_real)
    var_daily = bond_data_real_with_y0.groupby('date').apply(lambda x : (x['yield'].tail(1) - x['yield'].head(1))/((x['time'].tail(1)-x['time'].head(1))/timedelta(seconds=1)))
    
    variance_of_yield_changes = get_variance(bond_data_real_with_y0)
    print(variance_of_yield_changes)
    volatility = variance_of_yield_changes**(1/2)

In [ ]:
bond_data_real = trade_data_TB[trade_data_TB['trade_id'] == '190006.IB' ]
bond_data_real = bond_data_real.reset_index(drop = True)

In [ ]:

bond_data_real_with_y0 = add_y0(bond_data_real)

In [ ]:
bond_data_real.to_csv('a.csv')

In [ ]:
bond_data_real_with_y0.to_csv('b.csv')

In [ ]:
tail_data = get_tail_data(IB_190006)
tail_data = tail_data.reset_index(drop = True)

var_daily = adjust_first_day(IB_190006)
a = add_y0(IB_190006)
variance_of_yield_changes = get_variance(a)
volatility = variance_of_yield_changes**(1/2)

In [ ]:
variance_of_yield_changes[np.isinf(variance_of_yield_changes)] = np.nan
variance_of_yield_changes

In [ ]:
IB_190006_real = pd.read_excel('190006_real.xlsx')
IB_190006_real[test_type] = IB_190006_real[test_type]/100
rename_real_data(IB_190006_real)
IB_190006_real = IB_190006_real.drop_duplicates(['fitTime'])
IB_190006_real = IB_190006_real.drop('index',axis = 1)
IB_190006_real = IB_190006_real.reset_index(drop = True)

    
tail_data = get_tail_data(IB_190006_real)
tail_data = tail_data.reset_index(drop = True)
var_daily = adjust_first_day(IB_190006_real)

a = add_y0(IB_190006_real)
variance_of_yield_changes_real = get_variance(a)
volatility_real = variance_of_yield_changes_real**(1/2)
            


In [ ]:

variance_of_yield_changes_real[np.isinf(variance_of_yield_changes_real)] = np.nan
variance_of_yield_changes_real.mean()


In [ ]:
volatility_real.to_excel('volatility_real.xls')
volatility.to_excel('volatility_theoretical.xls')
variance_of_yield_changes.to_excel('variance_of_yield_changes_theo.xls')
variance_of_yield_changes_real.to_excel('variance_of_yield_changes_real.xls')